# NavigatorGPT - MongoDB Processing

In [ ]:
pip install openpyxl

In [291]:
import json
import pymongo
import pandas as pd
import pprint
import os
from pymongo import MongoClient, UpdateOne
from bson.objectid import ObjectId
from datetime import datetime

# Load the MongoDB connection string from the JSON file
PATH_TO_SECRET_JSON = '/home/jovyan/keys/mongodb_key.json'
with open(PATH_TO_SECRET_JSON) as f:
    MONGODB_URI = json.load(f)['connection_string']

# Extract the database name from the connection string
db_name = MONGODB_URI.split('/')[-1].split('?')[0]

# Create a MongoDB client using the connection string
if MONGODB_URI:
    client = MongoClient(MONGODB_URI)
    print("The MongoDB client has been initialized.")
else:
    print("Failed to initialize the MongoDB client.")

# Print the version of the pymongo package
pymongo_version = pymongo.__version__
print(f"The version of the pymongo package is {pymongo_version}")

# Connect to the specified MongoDB database and collection
db = client[db_name]
collection = db.attractions

The MongoDB client has been initialized.
The version of the pymongo package is 4.4.1


In [214]:
cursor = collection.find(
    {
        "loc.geo": {
            "$geoWithin": { 
                "$box": [
                   [-74.011941, 40.719226], [-74.000941, 40.709226]
                ] 
            }
        }
    },
    {"_id": 1, "title": 1}  # This is the projection parameter
)

results = []
for doc in cursor:
    results.append(doc)

# Create a DataFrame
df = pd.DataFrame(results)

markdown_table = df.to_markdown()

print(markdown_table)

|     | _id                      | title                                                                    |
|----:|:-------------------------|:-------------------------------------------------------------------------|
|   0 | 53668a995c959622000890a9 | Balloon Flower (Red)                                                     |
|   1 | 53668a995c959622000890b1 | Dreaming of Far Away Places: The Ships Come to Washington Market         |
|   2 | 53668a995c959622000890b7 | Five in One                                                              |
|   3 | 54fbf93293a2b1e1e52db607 | Double Check                                                             |
|   4 | 5520244d610bf1c45d848a37 | New York in Its Infancy                                                  |
|   5 | 55220b33610bf1c45d848a3c | Civic Fame                                                               |
|   6 | 55220d6d610bf1c45d848a3f | Entrance to Municipal Office Building                                    |
|   7 | 55

### CSV Processing

In [59]:
# Create a directory if it doesn't exist
output_directory = "../downloads/csv/"
os.makedirs(output_directory, exist_ok=True)

# Generate the Excel file name with current date and time
current_datetime = datetime.now().strftime("%Y_%m_%d_%H_%M_%S")
csv_filename = f"attractions_{current_datetime}.csv"
csv_filepath = os.path.join(output_directory, csv_filename)

# Save the DataFrame to an CSV file
df.to_csv(csv_filepath, index=False)

In [ ]:
# Find the document with the specified Objectid in the 'attractions' collection
document = collection.find_one({"_id": ObjectId("6450c55c3879cf70e53ff044")})

# Print the retrieved document
pprint.pprint(document)


**Description:**
We have a dataset of attractions with specific fields to be updated in the database. Below is the dataset:

In [284]:
data = [
    {
        "id": "6432ea18a90bce6b6c83bacd",
        "landmark.lpNumber": "LP-00597",
        "landmark.landmarkType": "Individual Landmark",
        "landmark.designationDate": {
            "$date": "1970-08-18T00:00:00"
        }
    },
    {
        "id": "6451f7e53879cf70e53ff088",
        "landmark.lpNumber": "LP-00599",
        "landmark.landmarkType": "Individual Landmark",
        "landmark.designationDate": {
            "$date": "1970-03-31T00:00:00"
        }
    },
]



In [285]:
data[:5]

[{'id': '6432ea18a90bce6b6c83bacd',
  'landmark.lpNumber': 'LP-00597',
  'landmark.landmarkType': 'Individual Landmark',
  'landmark.designationDate': {'$date': '1970-08-18T00:00:00'}},
 {'id': '6451f7e53879cf70e53ff088',
  'landmark.lpNumber': 'LP-00599',
  'landmark.landmarkType': 'Individual Landmark',
  'landmark.designationDate': {'$date': '1970-03-31T00:00:00'}}]

In [341]:
# Read the JSON array from the file
with open("../data/attractions/Location-Import_2023_09_30_19_48_58.json", "r") as file:
    data = json.load(file)
    
data[:5]

[{'id': '6487fdc23ee0be65782261ba',
  'landmark.lpNumber': 'LP-01133',
  'landmark.landmarkType': 'Individual Landmark',
  'landmark.designationDate': {'$date': '1981-02-03T00:00:00'}},
 {'id': '6487fc133ee0be65782261b8',
  'landmark.lpNumber': 'LP-01134',
  'landmark.landmarkType': 'Individual Landmark',
  'landmark.designationDate': {'$date': '1981-02-03T00:00:00'}},
 {'id': '6487e7313ee0be65782261b2',
  'landmark.lpNumber': 'LP-01135',
  'landmark.landmarkType': 'Individual Landmark',
  'landmark.designationDate': {'$date': '1981-08-11T00:00:00'}},
 {'id': '648b67a87c55b27c2c708030',
  'landmark.lpNumber': 'LP-01137',
  'landmark.landmarkType': 'Individual Landmark',
  'landmark.designationDate': {'$date': '1981-08-11T00:00:00'}},
 {'id': '648b69f47c55b27c2c708031',
  'landmark.lpNumber': 'LP-01138',
  'landmark.landmarkType': 'Individual Landmark',
  'landmark.designationDate': {'$date': '1981-08-11T00:00:00'}}]

**Description:**
Now, we'll loop through the dataset and update the records in the MongoDB collection.

In [342]:
from datetime import datetime

# Initialize counters and operations list
totalRecords = len(data)
recordsUpdated = 0
operations = []

# Prepare bulk update operations for each record in data
for item in data:
    update_fields = {}

    # Iterate over all keys in the item
    for key, value in item.items():
        # Skip 'id' key
        if key == 'id':
            continue

        # Check if the value is a dictionary (nested dictionary in MongoDB)
        if isinstance(value, dict):
            for sub_key, sub_value in value.items():
                # If the sub_key is "$date", convert the value to datetime
                if sub_key == "$date":
                    sub_value = datetime.fromisoformat(sub_value)
                    key = key.replace(".$date", "")
                update_fields[f"{key}"] = sub_value
        else:
            update_fields[key] = value

    # Create the update operation
    operations.append(
        UpdateOne(
            {"_id": ObjectId(item['id'])},
            {"$set": update_fields},
            upsert=True
        )
    )

# Execute the update operations in bulk
result = collection.bulk_write(operations)

# Calculate the total records updated
recordsUpdated = result.modified_count + len(result.upserted_ids)

# Output the results
print(f"Total records: {totalRecords}")
print(f"Records updated: {recordsUpdated}")


InvalidId: '554c0866083cd88a44265d1c ' is not a valid ObjectId, it must be a 12-byte input or a 24-character hex string

**Description:**
Print out the results, showing the total number of records processed and the number of records updated.

In [257]:
print(f"Total records: {totalRecords}")
print(f"Records updated: {recordsUpdated}")

Total records: 58
Records updated: 58
